In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Assuming you have a dataframe with two columns: 'image_path' and 'age'
# Load your dataset
data = pd.read_csv('/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv')

# Preprocess images
def preprocess_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    return img

# Apply preprocessing to all images in the dataset
data['image'] = data['file_id'].apply(preprocess_image)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['image'], data['age'], test_size=0.2, random_state=42)

# Convert age labels to categorical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Define the base model
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Add custom top layers
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='linear'))  # Output layer for regression

# Freeze the base model layers
base_model.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

# Define callbacks
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train_encoded, 
                    validation_data=(X_test, y_test_encoded), 
                    epochs=20, 
                    batch_size=32, 
                    callbacks=[checkpoint, early_stopping])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test_encoded)
print("Mean Absolute Error:", mae)

# Save the model
model.save("age_prediction_model.h5")

[ WARN:0@259.389] global loadsave.cpp:248 findDecoder imread_('image_13678.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
